<a href="https://colab.research.google.com/github/dawidkubicki/predict_customers_retention_ann/blob/main/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [57]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [58]:
#from google.colab import drive
#drive.mount('/content/drive')

In [59]:
#root_path = '/content/gdrive/My Drive/Colab Notebooks/my_files_dp_az/ann/'

import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/my_files_dp_az/ann/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'WORKING_DIRECTORY_JUST_TO_TEST_FILE.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab Notebooks/my_files_dp_az/ann/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [60]:
tf.__version__

'2.3.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [61]:
dataset = pd.read_csv('Churn_Modelling.csv')
#let's take all the columns without the last column and first 3 of them
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X[0])
#print(y)

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]


### Encoding categorical data

Label Encoding the "Gender" column

In [62]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[0])

[619 'France' 0 42 2 0.0 1 1 1 101348.88]


One Hot Encoding the "Geography" column

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


### Splitting the dataset into the Training set and Test set

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [65]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [66]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [67]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu', use_bias=True))

### Adding the second hidden layer

In [68]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu', use_bias=True))

### Adding the output layer

In [69]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [70]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [71]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5531 - accuracy: 0.7908
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4741 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4445 - accuracy: 0.8104
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4312 - accuracy: 0.8159
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4231 - accuracy: 0.8176
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4167 - accuracy: 0.8223
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4112 - accuracy: 0.8234
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4061 - accuracy: 0.8249
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4003 - accuracy: 0.8265
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3956 - accura

In [72]:
test_loss, test_acc = ann.evaluate(X_test, y_test)

63/63 [==============================] - 0s 929us/step - loss: 0.3317 - accuracy: 0.8640


## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [73]:
#predict array always has to be 2 deep (double [[]])
#[0.0 0.0 1.0 667 0 34 5 0.0 2 1 0 163830.64]
print(ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [74]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [76]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_pred, y_test)

[[1525   70]
 [ 202  203]]


0.864